In [4]:
import os, sys
sys.path.append(os.path.dirname("../models"))
from models.classifiers import SplitVotingEnsemble, SplitVotingEnsembleLGBM
from lightgbm import LGBMClassifier 
from sklearn.ensemble import RandomForestClassifier
import pandas as pd
import numpy as np
from HELPpy.models.prediction import VotingEnsembleLGBM
#clf = RandomForestClassifier()
clf = LGBMClassifier(n_estimators=200)
svclf = SplitVotingEnsemble(clf, n_voters=10, verbose=True)
#svclf = VotingEnsembleLGBM(verbose=True)
from sklearn.datasets import load_iris
data = load_iris()
#X = pd.DataFrame(data.data, columns=data.feature_names)
#Y = pd.DataFrame(data['target'], columns=['target'])
X = pd.read_csv("../../data/Kidney_BIO.csv", index_col=0)
#Y = pd.read_csv("../../data/Kidney_HELP.csv", index_col=0).replace({'sNE':'NE', 'aE':'NE', 'E':'E'})
Y = pd.read_csv("../../data/Kidney_HELP.csv", index_col=0).replace({'sNE':0, 'aE':0, 'E':1})
shdgenes = np.intersect1d(X.index.values, Y.index.values)
X = X.loc[shdgenes]
Y = Y.loc[shdgenes]
preds = svclf.fit(X,Y).predict(X)
svclf.get_params()

Majority 0 15994, 2nd major 1 1242


{'clf__boosting_type': 'gbdt',
 'clf__class_weight': None,
 'clf__colsample_bytree': 1.0,
 'clf__importance_type': 'split',
 'clf__learning_rate': 0.1,
 'clf__max_depth': -1,
 'clf__min_child_samples': 20,
 'clf__min_child_weight': 0.001,
 'clf__min_split_gain': 0.0,
 'clf__n_estimators': 200,
 'clf__n_jobs': -1,
 'clf__num_leaves': 31,
 'clf__objective': None,
 'clf__random_state': None,
 'clf__reg_alpha': 0.0,
 'clf__reg_lambda': 0.0,
 'clf__silent': 'warn',
 'clf__subsample': 1.0,
 'clf__subsample_for_bin': 200000,
 'clf__subsample_freq': 0,
 'clf': LGBMClassifier(n_estimators=200),
 'n_jobs': -1,
 'n_voters': 10,
 'random_state': 42,
 'verbose': True,
 'voting': 'soft'}

In [51]:
svclf.predict_proba(X.loc[['AASDH']]), svclf.predict(X.loc[['AASDH']])


(array([[9.99999188e-01, 8.12057013e-07]]), array([0]))

In [11]:
from sklearn.metrics import balanced_accuracy_score
balanced_accuracy_score(Y, preds)

0.9220957859197199

In [50]:
from HELPpy.models.prediction import k_fold_cv, VotingEnsembleLGBM
clf = LGBMClassifier(n_estimators=200)
svclf = SplitVotingEnsemble(clf, verbose=True)
svclf = SplitVotingEnsembleLGBM(n_estimators=200, verbose=True)
#svclf = VotingEnsembleLGBM(n_estimators=200, verbose=True)
df_scores, scores, predictions = k_fold_cv(X, Y, svclf, n_splits=5, seed=0, show_progress=True, verbose=True)
df_scores

{0: 0, 1: 1}
label
0        15994
1         1242
Name: count, dtype: int64



5-fold:   0%|          | 0/5 [00:00<?, ?it/s]

Majority 1 12795, 2nd major 0 993
Majority 1 12795, 2nd major 0 994
Majority 1 12795, 2nd major 0 994
Majority 1 12795, 2nd major 0 994
Majority 1 12796, 2nd major 0 993


,measure
ROC-AUC,0.8979±0.0080
Accuracy,0.9296±0.0007
BA,0.5117±0.0059
Sensitivity,0.0233±0.0118
Specificity,1.0000±0.0000
MCC,0.1435±0.0370
CM,"[[15994, 0], [1213, 29]]"


In [46]:
predictions

,label,prediction,probabilities
gene,,,
A1CF,0,0,0.999246
A2M,0,1,0.492280
AAGAB,0,0,0.665028
AARD,0,0,0.999324
AASDH,0,1,0.476863
...,...,...,...
ZSWIM7,0,0,0.983311
ZSWIM8,0,1,0.316870
ZXDA,0,0,0.996721


In [7]:
svclf.get_params()

{'boosting_type': 'gbdt',
 'learning_rate': 0.1,
 'n_estimators': 200,
 'n_jobs': -1,
 'n_voters': 13,
 'random_state': 42,
 'verbose': True,
 'voting': 'soft'}